In [9]:
import cv2
import numpy as np


cap = cv2.VideoCapture(0)

yawn_counter = 0
yawn_threshold = 20 

def calculate_mouth_aspect_ratio(mouth):
    x, y, w, h = cv2.boundingRect(mouth)
    aspect_ratio = w / h
    return aspect_ratio

while True:
    # Capture frame
    ret, frame = cap.read()

    # Convert the frame to RGB 
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Define the color range for detecting the mouth in RGB
    lower_color = np.array([240, 120, 120], dtype=np.uint8)
    upper_color = np.array([255, 245, 245], dtype=np.uint8)

    # Create  mask for the mouth region
    mouth_mask = cv2.inRange(rgb_frame, lower_color, upper_color)

    # Find contours in the mask
    contours, _ = cv2.findContours(mouth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 100:
            mouth_aspect_ratio = calculate_mouth_aspect_ratio(contour)

            # Check for yawn based on mouth aspect ratio
            if mouth_aspect_ratio > yawn_threshold:
                yawn_counter += 1
                cv2.putText(frame, f'Yawn Count: {yawn_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                            cv2.LINE_AA)

            cv2.drawContours(frame, [contour], -1, (0, 255, 0), 2)

    cv2.imshow('Yawn Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
